# "Cross-validation" des modèles

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Hyper-parametres
learning_rate = 0.04
num_epochs = 400
N = 200            #N = 150 a la base
N_cv = 40
seed = 42
batch_size = N//4
#batch_size = N//2
N_test = 0
N_test = 2000      #N_test = 1000 à la base
N_scan = 9
bias = True


p0 = 0.02          #p0 = 0.05 à la base
theta0 = 0
wt = np.pi/20      
theta_std = np.pi/6



## Creer des données synthetiques 

In [ ]:
def get_data(
            N = N,
            p0 = p0,
            theta0 = theta0,
            wt = wt,
            theta_std = theta_std,
            seed=seed):
    np.random.seed(42)
    theta = np.random.randn(N)*theta_std
    a = (theta-theta0)/wt
    p = 1/(1+np.exp(-a))
    
    p = p0/2 + (1-p0) * p #add lapse rate
    y = np.random.rand(N) < p #generate data
    return theta, p, y

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
torch.set_default_tensor_type('torch.DoubleTensor')
criterion = torch.nn.BCELoss()

class LogisticRegressionModel(torch.nn.Module):
    def __init__(self, bias=True, logit0=-2): #-2 ?
        super(LogisticRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(1, 1, bias=bias)    
        self.logit0 = torch.nn.Parameter(logit0*torch.ones(1))

    def forward(self, theta):
        out = self.logit0.sigmoid()/2 + (1-self.logit0.sigmoid())*self.linear(theta).sigmoid()
        return out
        

def fit_data(theta, y, 
                learning_rate =learning_rate,
                num_epochs = num_epochs,
                batch_size = batch_size,
                verbose=False):

    logistic_model = LogisticRegressionModel()

    labels = torch.Tensor(y[:, None])
    Theta = torch.Tensor(theta[:, None])

    loader = DataLoader(TensorDataset(Theta, labels), batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(logistic_model.parameters(), lr=learning_rate/len(loader))
    
    for epoch in range(int(num_epochs)):
        losses = []
        for Theta_, labels_ in loader:
            optimizer.zero_grad()
            outputs = logistic_model(Theta_)
            loss = criterion(outputs, labels_)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        if verbose and (epoch % (num_epochs//32) == 0) : 
            print(f"Iteration: {epoch} - Loss: {np.mean(losses):.5f}")

    logistic_model.eval()
    return logistic_model, np.mean(losses)

In [ ]:
torch.sigmoid(torch.tensor([-3.]))

In [ ]:
%%timeit
theta, p, y = get_data()

In [ ]:
theta, p, y = get_data()
logistic_model, loss = fit_data(theta, y,verbose=True) 

In [ ]:
with torch.no_grad():
    if bias: print('bias', logistic_model.linear.bias.item())
    print('slope', logistic_model.linear.weight.item())    
    print('p0', torch.sigmoid(logistic_model.logit0).item())


In [ ]:
print('loss=', loss)
plt.figure(figsize = (9,6)) 
plt.scatter(theta, p, s=4, color = 'r', label='proba cachées')
plt.scatter(theta, y, s=1, alpha=.1, color = 'b', label='données')
x_values = np.linspace(-1.5, 1.50, 100)[:, None]
y_values = logistic_model(torch.Tensor(x_values)).detach().numpy()
plt.plot(x_values, y_values, 'g', alpha=.7, lw=3, label='proba prédites')
plt.xlabel(r'$\theta$')
plt.yticks([0.,1.],['Left', 'Right']);
plt.legend();


In [ ]:
%%timeit
logistic_model, loss = fit_data(theta, y, verbose=False)

## validation

In [ ]:
theta, p, y = get_data() # nouvelles données 

labels = torch.Tensor(y[:, None])
Theta = torch.Tensor(theta[:, None])
outputs = logistic_model(Theta)
loss = criterion(outputs, labels)
print('loss=', loss)
plt.figure(figsize = (8,6)) 
plt.scatter(theta, p, s=4, color = 'r', label='proba cachées')
plt.scatter(theta, y, s=1, alpha=.1, color = 'b', label='données')
x_values = np.linspace(-1.5, 1.50, 100)[:, None]
y_values = logistic_model(torch.Tensor(x_values)).detach().numpy()
plt.plot(x_values, y_values, 'g', alpha=.7, lw=3, label='proba prédites')
plt.xlabel(r'$\theta$')
plt.yticks([0.,1.],['Left', 'Right']);
plt.legend();

## influence du nombre de trials

In [ ]:
Ns = np.logspace(1, 3, N_scan, base=10)

Ns_, losses, loss_Ps, loss_P0s = [], [], [], []

for N_ in Ns:
    for i_CV in range(N_cv):
        theta, p, y = get_data(N=int(N_), seed=seed+i_CV)
        logistic_model, loss = fit_data(theta, y, verbose=False)
        
        if N_test>0: theta, p, y = get_data(N=N_test) # nouvelles données 
        labels = torch.Tensor(y[:, None])
        Theta = torch.Tensor(theta[:, None])
        P = torch.Tensor(p[:, None])
        
        outputs = logistic_model(Theta)
        loss = criterion(outputs, labels).item()
    
        loss_P = criterion(outputs, P).item() 
        loss_P0 = criterion(P, P).item()
        
        if i_CV==0: print(f"N: {int(N_)}, Loss: {loss:.5f}, loss_P: {loss_P:.5f}, loss_P0: {loss_P0:.5f}")
        loss_P0s.append(loss_P0)
        Ns_.append(N_)
        loss_Ps.append(loss_P)
        losses.append(loss)
    

In [ ]:
#plot

fig, ax = plt.subplots(figsize = (15, 8)) 
ax.scatter(Ns_, losses, alpha=3/N_cv, label='loss')
ax.scatter(Ns_, loss_Ps, alpha=3/N_cv, label='loss_P')
ax.plot(Ns_, loss_P0s, label='loss_P0')

ax.set_xlabel(' # trials')
ax.set_ylabel(' Loss ')
ax.set_xscale('log')

## influence du nombre du learning rate


In [ ]:
learning_rates = learning_rate * np.logspace(-1, 1, N_scan, base=10)
learning_rates_, losses, loss_Ps, loss_P0s = [], [], [], []
for learning_rate_ in learning_rates:
    for i_CV in range(N_cv):
        theta, p, y = get_data(seed=seed+i_CV)
        logistic_model, loss = fit_data(theta, y, learning_rate=learning_rate_, verbose=False)

        if N_test>0: theta, p, y = get_data(N=N_test) # nouvelles données 
        labels = torch.Tensor(y[:, None])
        Theta = torch.Tensor(theta[:, None])
        outputs = logistic_model(Theta)

        loss = criterion(outputs, labels).item()
        loss_P = criterion(outputs, torch.Tensor(p[:, None])).item()
        loss_P0 = criterion(torch.Tensor(p[:, None]), torch.Tensor(p[:, None])).item()
        if i_CV==0: 
            print(f"learning_rate: {learning_rate_:.5f}, Loss: {loss:.5f}, loss_P: {loss_P:.5f}, loss_P0: {loss_P0:.5f}")
        learning_rates_.append(learning_rate_)
        loss_P0s.append(loss_P0)
        loss_Ps.append(loss_P)
        losses.append(loss)

In [ ]:
#influence du learning rate sur loss

fig, ax = plt.subplots(figsize = (15, 8)) 
ax.scatter(learning_rates_, losses, alpha=3/N_cv, label='loss')
ax.scatter(learning_rates_, loss_Ps, alpha=3/N_cv, label='loss_P')
ax.plot(learning_rates_, loss_P0s, label='loss_P0')
ax.set_xlim(np.min(learning_rates_), np.max(learning_rates_))

ax.set_xlabel('learning_rate')
ax.set_ylabel(' Loss ')
ax.set_xscale('log')
ax.legend(loc='best');

## influence du nombre d'epochs

In [ ]:
num_epochss = num_epochs * np.logspace(-1, 1, N_scan, base=10)
num_epochss_, losses, loss_Ps, loss_P0s = [], [], [], []
for num_epochs_ in num_epochss:
    for i_CV in range(N_cv):
        theta, p, y = get_data(seed=seed+i_CV)
        logistic_model, loss = fit_data(theta, y, num_epochs=int(num_epochs_), verbose=False)
        
        if N_test>0: theta, p, y = get_data(N=N_test) # nouvelles données 
        labels = torch.Tensor(y[:, None])
        Theta = torch.Tensor(theta[:, None])
        outputs = logistic_model(Theta)

        loss = criterion(outputs, labels).item()
        loss_P = criterion(outputs, torch.Tensor(p[:, None])).item()
        loss_P0 = criterion(torch.Tensor(p[:, None]), torch.Tensor(p[:, None])).item()
        if i_CV==0: 
            print(f"num_epochs: {int(num_epochs_)}, Loss: {loss:.5f}, loss_P: {loss_P:.5f}, loss_P0: {loss_P0:.5f}")
        num_epochss_.append(num_epochs_)
        loss_P0s.append(loss_P0)
        loss_Ps.append(loss_P)
        losses.append(loss)

In [ ]:
# influence du nbr d'epochs sur loss 
fig, ax = plt.subplots(figsize = (15, 8)) 
ax.scatter(num_epochss_, losses, alpha=3/N_cv, label='loss')
ax.scatter(num_epochss_, loss_Ps, alpha=3/N_cv, label='loss_P')
ax.plot(num_epochss_, loss_P0s, label='loss_P0')

ax.set_xlabel(' # epochs')
ax.set_ylabel(' Loss ')
ax.set_xscale('log')
ax.legend(loc='best');

## influence de la taille du minibatch

In [ ]:
batch_sizes = N * np.logspace(-3, 0, N_scan, base=2)
batch_sizes_, losses, loss_Ps, loss_P0s = [], [], [], []
for batch_size_ in batch_sizes:
    for i_CV in range(N_cv):
        theta, p, y = get_data(seed=seed+i_CV)
        logistic_model, loss = fit_data(theta, y, batch_size=int(batch_size_), verbose=False)
        
        if N_test>0: theta, p, y = get_data(N=N_test) # nouvelles données 
        labels = torch.Tensor(y[:, None])
        Theta = torch.Tensor(theta[:, None])
        outputs = logistic_model(Theta)

        loss = criterion(outputs, labels).item()
        loss_P = criterion(outputs, torch.Tensor(p[:, None])).item()
        loss_P0 = criterion(torch.Tensor(p[:, None]), torch.Tensor(p[:, None])).item()
        if i_CV==0: 
            print(f"batch_size: {int(batch_size_)}, Loss: {loss:.5f}, loss_P: {loss_P:.5f}, loss_P0: {loss_P0:.5f}")
        batch_sizes_.append(batch_size_)
        loss_P0s.append(loss_P0)
        loss_Ps.append(loss_P)
        losses.append(loss)

In [ ]:
# influence de la taille du minibatch sur loss 

fig, ax = plt.subplots(figsize = (15, 8)) 
ax.scatter(batch_sizes_, losses, alpha=3/N_cv, label='loss')
ax.scatter(batch_sizes_, loss_Ps, alpha=3/N_cv, label='loss_P')
ax.plot(batch_sizes_, loss_P0s, label='loss_P0')

ax.set_xlabel(' batch_size')
ax.set_ylabel(' Loss ')
ax.set_xscale('log')
ax.legend(loc='best');

## Comparaison données générées/données prédites

## p0 prediction/generate I


In [ ]:

p0s = np.linspace(0, 1, 50)

preds = {'torch':{'theta0_preds':[], 'wt_preds':[], 'p0_preds':[]}, 
         'sklearn':{'theta0_preds':[], 'wt_preds':[], 'p0_preds':[]}}

for p0_ in p0s:
    seed_ += 1
    theta, p, y = get_data(p0=p0_, seed=seed_)
    
    # fir with our method
    logistic_model, loss = fit_data(theta, y, verbose=False)
    p0_pred =  torch.sigmoid(logistic_model.logit0).item()     
    # print('bias', logistic_model.linear.bias.item())
    # print('slope', logistic_model.linear.weight.item())    
    # print('p0', torch.sigmoid(logistic_model.logit0).item())
    
    
    
    p0_preds.append(p0_pred)
    # print(f"p0 : {p0_pred: .5f}")

    # TODO fit with sklearn



In [ ]:
# TODO : show three panels

fig, axs= plt.subplots(1, 3, figsize = (15, 8)) 
axs[0].scatter(p0s, p0_s)
plt.xlabel('p0 réel')
plt.ylabel('p0 prédit')


## theta0 prediction/generate 

In [ ]:
theta0s = np.random.randn(50)
theta0_preds = []

for theta0_ in theta0s:
    
    theta, p, y = get_data(N=int(N_), seed=seed+i_CV)
    logistic_model, loss = fit_data(theta, y, verbose=False)
        
    theta0_pred = logistic_model.linear.bias.item()
    theta0_s.append(theta0_pred)
        
    print(f" theta0: {theta0_pred:.5f}")
    

In [ ]:
plt.figure(figsize = (8,8)) 
plt.scatter(theta0s, theta0_s)
plt.xlabel('theta réel')
plt.ylabel('theta prédit')


## wt prediction/generate 

In [ ]:
wts = np.logspace(-1, 0.1, 50, base=10)
wt_s = []

for wt_ in wts:
    
    theta, p, y = get_data(N=int(N_), seed=seed+i_CV)
    logistic_model, loss = fit_data(theta, y, verbose=False)
        
    if N_test>0: theta, p, y = get_data(N=N_test) # nouvelles données 
        
    wt_pred = logistic_model.linear.weight.item()
        
    wt_s.append(wt_pred)
    print(f" wt:{wt_pred:.5f}")    

In [ ]:
plt.figure(figsize = (8,8)) 
plt.scatter(wts, wt_s)
plt.xlabel('pente réelle')
plt.ylabel('pente prédite')


### Tentative II

In [ ]:
# p0 prediction/generate I

p0s = np.linspace(0,0.03,50)
p0_s = []


for p0_ in p0s:
    
    theta, p, y = get_data(N=int(N_), seed=seed+i_CV)
    logistic_model, loss = fit_data(theta, y, verbose=False)

    if N_test>0: theta, p, y = get_data(N=N_test) # nouvelles données 
    p0_pred =  torch.sigmoid(logistic_model.logit0).item()     
        
    p0_s.append( p0_pred)
    print(f"p0 : {p0_pred: .5f}")

plt.figure(figsize = (8,8)) 
plt.scatter(p0s, p0_s)
plt.xlabel('p0 réel')
plt.ylabel('p0 prédit')


In [ ]:
#theta0 prediction/generate 

theta0s = np.random.randn(50) #hum... pas terrible
theta0_s = []

for theta0_ in theta0s:
    
    theta, p, y = get_data(N=int(N_), seed=seed+i_CV)
    logistic_model, loss = fit_data(theta, y, verbose=False)

    if N_test>0: theta, p, y = get_data(N=N_test) # nouvelles données 
        
    theta0_pred = logistic_model.linear.bias.item()
    theta0_s.append(theta0_pred)
        
    print(f" theta0: {theta0_pred:.5f}")
    
plt.figure(figsize = (8,8)) 
plt.scatter(theta0s, theta0_s)
plt.xlabel('theta réel')
plt.ylabel('theta prédit')


In [ ]:
#wt prediction/generate 

wts = np.logspace(-1, 1, 50, base=10)
wt_s = []

for wt_ in wts:
    
    theta, p, y = get_data(N=int(N_), seed=seed+i_CV)
    logistic_model, loss = fit_data(theta, y, verbose=False)
        
    if N_test>0: theta, p, y = get_data(N=N_test) # nouvelles données 
        
    wt_pred = logistic_model.linear.weight.item()
        
    wt_s.append(wt_pred)
    print(f" wt:{wt_pred:.5f}") 
    
plt.figure(figsize = (8,8)) 
plt.scatter(wts, wt_s)
plt.xlabel('pente réelle')
plt.ylabel('pente prédite')
